Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (80000, 28, 28) (80000,)
Validation set (4000, 28, 28) (4000,)
Test set (4000, 28, 28) (4000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (80000, 784) (80000, 10)
Validation set (4000, 784) (4000, 10)
Test set (4000, 784) (4000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [25]:
###SGD
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  print(logits)
  l2regulate= tf.nn.l2_loss(logits)
  print(l2regulate)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)+1e-6*l2regulate)
  print(loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Tensor("add:0", shape=(128, 10), dtype=float32)
Tensor("L2Loss:0", shape=(), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)


In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.718613
Minibatch accuracy: 19.5%
Validation accuracy: 12.2%
Minibatch loss at step 500: 1.355079
Minibatch accuracy: 77.3%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 1.782970
Minibatch accuracy: 73.4%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 1.086437
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2000: 0.733362
Minibatch accuracy: 81.2%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 0.996418
Minibatch accuracy: 71.9%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 0.853069
Minibatch accuracy: 82.8%
Validation accuracy: 79.5%
Test accuracy: 85.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
###SGD
train_subsettt = train_dataset[0:1000,:]
train_sublabel = train_labels[0:1000,:]
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subsettt[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.158688
Minibatch accuracy: 14.1%
Validation accuracy: 17.8%
Minibatch loss at step 200: 0.795997
Minibatch accuracy: 85.2%
Validation accuracy: 68.7%
Minibatch loss at step 400: 0.197597
Minibatch accuracy: 96.9%
Validation accuracy: 68.7%
Minibatch loss at step 600: 0.091157
Minibatch accuracy: 99.2%
Validation accuracy: 69.0%
Minibatch loss at step 800: 0.055331
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 1000: 0.051629
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 1200: 0.037718
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1400: 0.032254
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 1600: 0.033336
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Minibatch loss at step 1800: 0.030272
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Minibatch loss at step 2000: 0.022581
Minibatch accuracy: 100.0%
Validation ac

ANS: training accuracy is 100%, but validation accuracy and test accuracy is around 70%.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [7]:
###SGD
train_subsettt = train_dataset[0:1000,:]
train_sublabel = train_labels[0:1000,:]
batch_size = 128
dddropout=[0.8]
num_steps = 2001
for dropouttt in dddropout:
    graph = tf.Graph()
    with graph.as_default():
      # Input data. 
      tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
      # Variables.
      weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
      biases = tf.Variable(tf.zeros([num_labels]))
      
      # Training computation.
      logits = tf.matmul(tf_train_dataset, weights) + biases
      logits = tf.nn.dropout(logits, dropouttt)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)
      valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
      test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
        
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_subsettt[offset:(offset + batch_size), :]
        batch_labels = train_sublabel[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropouttt}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict = {keep_prob:1.0}), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 23.126530
Minibatch accuracy: 11.7%
Validation accuracy: 9.9%
Minibatch loss at step 500: 1.280097
Minibatch accuracy: 51.6%
Validation accuracy: 67.2%
Minibatch loss at step 1000: 0.909136
Minibatch accuracy: 59.4%
Validation accuracy: 67.6%
Minibatch loss at step 1500: 0.876161
Minibatch accuracy: 60.2%
Validation accuracy: 68.5%
Minibatch loss at step 2000: 0.955561
Minibatch accuracy: 60.2%
Validation accuracy: 68.5%
Test accuracy: 75.1%
Initialized
Minibatch loss at step 0: 24.791967
Minibatch accuracy: 10.9%
Validation accuracy: 13.7%
Minibatch loss at step 500: 0.982232
Minibatch accuracy: 63.3%
Validation accuracy: 67.5%
Minibatch loss at step 1000: 0.775603
Minibatch accuracy: 66.4%
Validation accuracy: 67.8%
Minibatch loss at step 1500: 0.760224
Minibatch accuracy: 64.8%
Validation accuracy: 68.7%
Minibatch loss at step 2000: 0.607161
Minibatch accuracy: 69.5%
Validation accuracy: 69.4%
Test accuracy: 75.3%
Initialized
Minibatch loss at s

Initialized
Minibatch loss at step 0: 20.814812
Minibatch accuracy: 10.9%
Validation accuracy: 12.7%
Minibatch loss at step 500: 0.638551
Minibatch accuracy: 78.1%
Validation accuracy: 69.2%
Minibatch loss at step 1000: 0.446889
Minibatch accuracy: 85.2%
Validation accuracy: 70.2%
Minibatch loss at step 1500: 0.306119
Minibatch accuracy: 87.5%
Validation accuracy: 70.9%
Minibatch loss at step 2000: 0.303973
Minibatch accuracy: 89.1%
Validation accuracy: 70.6%
Test accuracy: 76.2%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---

In [38]:
###SGD
train_subsettt = train_dataset[0:10000,:]
train_sublabel = train_labels[0:10000,:]
batch_size = 128
dropouttt=1.0
graph = tf.Graph()
with graph.as_default():

  # Input data. 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.96, staircase=True)
  keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  ##weight1=shape(imagesize, middlelabel)
  ##weight2=shape(middlelabel, num_label)
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = tf.nn.dropout(logits, dropouttt)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [39]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_subsettt.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_subsettt[offset:(offset + batch_size), :]
    batch_labels = train_sublabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : dropouttt}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict = {keep_prob:1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict = {keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 23.826630
Minibatch accuracy: 8.6%
Validation accuracy: 15.1%
Minibatch loss at step 500: 1.546398
Minibatch accuracy: 57.8%
Validation accuracy: 73.5%
Minibatch loss at step 1000: 1.319362
Minibatch accuracy: 59.4%
Validation accuracy: 74.9%
Minibatch loss at step 1500: 1.794065
Minibatch accuracy: 55.5%
Validation accuracy: 75.0%
Minibatch loss at step 2000: 1.485711
Minibatch accuracy: 54.7%
Validation accuracy: 75.6%
Minibatch loss at step 2500: 1.729220
Minibatch accuracy: 47.7%
Validation accuracy: 75.8%
Minibatch loss at step 3000: 1.276979
Minibatch accuracy: 60.2%
Validation accuracy: 75.8%
Minibatch loss at step 3500: 1.379959
Minibatch accuracy: 55.5%
Validation accuracy: 76.3%
Minibatch loss at step 4000: 1.312955
Minibatch accuracy: 58.6%
Validation accuracy: 76.0%
Test accuracy: 82.8%
